In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
UNKNOWN = 0
EMPTY = 1
PLAYER = 2
WALL = 3
EXIT = 4
DOOR_RED = 5
KEY_RED = 6
DOOR_GREEN = 7
KEY_GREEN = 8
DOOR_BLUE = 9
KEY_BLUE = 10

colors = {
    UNKNOWN:    [  0,   0,   0],
    EMPTY:      [ 50,  50,  50],
    PLAYER:     [  0,   0, 255],
    WALL:       [147, 124,  93],
    EXIT:       [230, 217, 177],
    DOOR_RED:   [128,   0,   0],
    KEY_RED:    [255,   0,   0],
    DOOR_GREEN: [  0, 128,  0],
    KEY_GREEN:  [  0, 255,   0],
    DOOR_BLUE:  [  0,   0, 128],
    KEY_BLUE:   [  0,   0, 255],
}

In [ ]:
def get_map_image(map, height, width):
    map_img = np.zeros((height, width, 3), dtype=np.float32)
    for y in range(height):
        for x in range(width):
            map_img[y, x] = (np.array(colors[map[y, x]]) / 255).astype(np.float32)
    return map_img

def plot_map(map, height, width):
    map_img = get_map_image(map, height, width)
    plt.figure()
    plt.gca().set_axis_off()
    plt.tight_layout(pad=0)
    plt.imshow(map_img)
    plt.show()

In [ ]:
def get_distances_from(position: tuple[int,int]):
    global map, width, height

    distances = {}
    todo = []

    todo.append(position)
    distances[position] = 0

    def check_and_add(next):
        global map
        nonlocal distances, todo
        next_y, next_x = next
        next_dist = distances[next] if next in distances else np.inf
        if map[next_y,next_x] == EMPTY:
            if cur_dist+1 < next_dist:
                distances[next] = cur_dist+1
                todo.append(next)

    while todo:
        cur = todo[0]
        del todo[0]
        
        cur_y, cur_x = cur
        cur_dist = distances[cur]
        
        if cur_y > 0: check_and_add((cur_y-1, cur_x))
        if cur_y < height-1: check_and_add((cur_y+1, cur_x))
        if cur_x > 0: check_and_add((cur_y, cur_x-1))
        if cur_x < width-1: check_and_add((cur_y, cur_x+1))

    return distances

In [ ]:
height, width = 50, 50

# Standard map with edges
map = np.ones((height, width)) * WALL
# for x in range(width):
#     map[0, x] = WALL
#     map[height-1, x] = WALL
# for y in range(1, height-1):
#     map[y, 0] = WALL
#     map[y, width-1] = WALL

def create_room(center_y, center_x, block_height, block_width):
    global map, height, width
    
    left = center_x - block_width // 2
    right = left + block_width
    top = center_y - block_height // 2
    bottom = top + block_height
    
    for y in range(top, bottom+1):
        for x in range(left, right+1):
            map[y, x] = EMPTY

rooms = []

for _ in range(20):
    bw = np.random.randint(4, 15)
    bh = np.random.randint(4, 15)

    map_choice = np.ones_like(map)

    for y in range(0, 1 + bh//2):
        for x in range(width):
            map_choice[y, x] = 0

    for y in range(height - 1 - (bh - bh//2), height):
        for x in range(width):
            map_choice[y, x] = 0

    for x in range(0, 1 + bw//2):
        for y in range(height):
            map_choice[y, x] = 0

    for x in range(width - 1 - (bw - bw//2), width):
        for y in range(height):
            map_choice[y, x] = 0

    for y in range(height):
        for x in range(0, width):
            if map[y,x] != WALL and EXIT:
                for cy in range(y - (bh-bh//2) - 1, y + bh//2 + 2):
                    for cx in range(x - (bw-bw//2) - 1, x + bw//2 + 2):
                        if 0 <= cy < height and 0 <= cx < width:
                            map_choice[cy, cx] = 0
                    
    #plot_map(map_choice, height, width)

    I = np.argwhere(map_choice == 1)
    if len(I) == 0: continue
    by,bx = I[np.random.choice(len(I))]

    create_room(by, bx, bh, bw)
    rooms.append((by, bx, bh, bw))

rooms = np.array(rooms)

todo = list(range(len(rooms)))

# start with top left room
current_index = np.argmin(np.array([math.sqrt(rooms[t][0]**2 + rooms[t][1]**2) for t in todo]))
current = todo[current_index]

while len(todo) > 1:
    current_y, current_x, _, _ = rooms[current]
    todo.remove(current)

    closest_indices = np.argsort([math.sqrt((rooms[t][0]-current_y)**2 + (rooms[t][1]-current_x)**2) for t in todo])[:2]
    next_index = np.random.choice(closest_indices)
    next = todo[next_index]

    # connect
    next_y, next_x, _, _ = rooms[next]

    if abs(next_x - current_x) > abs(next_y - current_y):
        if next_x != current_x:
            dir = np.sign(next_x - current_x)
            for x in range(current_x, next_x+dir, dir):
                map[current_y, x] = EMPTY
                map[current_y-dir, x] = EMPTY
        if next_y != current_y:
            dir = np.sign(next_y - current_y)
            for y in range(current_y, next_y+dir, dir):
                map[y, next_x] = EMPTY
                map[y, next_x-dir] = EMPTY
    else:
        if next_y != current_y:
            dir = np.sign(next_y - current_y)
            for y in range(current_y, next_y+dir, dir):
                map[y, current_x] = EMPTY
                map[y, current_x-dir] = EMPTY
        if next_x != current_x:
            dir = np.sign(next_x - current_x)
            for x in range(current_x, next_x+dir, dir):
                map[next_y, x] = EMPTY
                map[next_y-dir, x] = EMPTY

    current = next

# place player in top left room
top_left = np.argmin(np.array([math.sqrt(rx*rx + ry*ry) for ry,rx,_,_ in rooms]))
ry, rx, rh, rw = rooms[top_left]
py = ry
px = rx
map[py, px] = PLAYER

# place exit in bottom right room
bottom_right = np.argmax(np.array([math.sqrt(rx*rx + ry*ry) for ry,rx,_,_ in rooms]))
ry, rx, rh, rw = rooms[bottom_right]
ey = ry + (rh - rh//2)
ex = rx + (rw - rw//2) + 1
map[ey, ex] = EXIT

plot_map(map, height, width)

In [ ]:
def add_distances_from(distances:dict, position:tuple[int,int]):
    distances_extra = get_distances_from(position)

    distances_combined = {}
    for k in distances.keys():
        d = distances[k]
        if k in distances_extra:
            de = distances_extra[k]
            d += de
        distances_combined[k] = d
    return distances_combined

In [ ]:
def plot_distances(distances):
    global height, width
    D = np.zeros((height, width), np.float32)
    for y in range(height):
        for x in range(width):
            p = (y,x)
            if p in distances:
                D[y,x] = distances[p]
            else:
                D[y,x] = np.inf
                
    plt.figure()
    plt.imshow(D)

In [ ]:
distances = get_distances_from((ey,ex))
distances = add_distances_from(distances, (py,px))
plot_distances(distances)

(far_y,far_x),far_dist = sorted(distances.items(), key=lambda kvp: kvp[1])[-1]
map[far_y, far_x] = KEY_RED
plot_map(map, height, width)

distances = add_distances_from(distances, (far_y,far_x))
plot_distances(distances)

(far_y,far_x),far_dist = sorted(distances.items(), key=lambda kvp: kvp[1])[-1]
map[far_y, far_x] = KEY_GREEN
plot_map(map, height, width)
